In [4]:
#!{sys.executable} -m pip install matplotlib

In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import ipywidgets as widgets

In [134]:
def filter_df(df_orig, col, arg, op='=='):
    df = df_orig.copy()
    
    if op == '==':
        df = df.loc[df[col]==arg]
    elif op == '<':
        df = df.loc[df[col]<arg]
    elif op == '>':
        df = df.loc[df[col]>arg]
    elif op == '<=':
        df = df.loc[df[col]<=arg]
    elif op == '>=':
        df = df.loc[df[col]>=arg]
    elif op == '!=':
        df = df.loc[df[col]!=arg]
    else:
        print('op type not supported!')
    
    return df

In [135]:
config_df = pd.read_csv("../cmake-build-debug/config_log.txt", sep=",", header=0)
display(config_df)
config_dict = config_df.to_dict(orient='list')


sim_df = pd.read_csv("../cmake-build-debug/sim_log.txt", sep=",", header=0)
display(sim_df.head())

sampling_df = pd.read_csv("../cmake-build-debug/sampling_log.txt", sep=",", header=0)
display(sampling_df.head())

,sim_time,rollouts,horizon,obstacle_cost,obstacle_rad_0,obstacle_rad_1,obstacle_rad_2,obstacle_rad_3,obstacle_pos_0_x,obstacle_pos_0_y,obstacle_pos_1_x,obstacle_pos_1_y,obstacle_pos_2_x,obstacle_pos_2_y,obstacle_pos_3_x,obstacle_pos_3_y,target_state_0,target_state_1,Unnamed: 18
0,150,20,15,10000,10,5,10,5,5,20,55,15,40,20,80,40,100,50,NaN


,simstep,state_0,state_1,state_2,state_3,state_4,state_5,Unnamed: 7
0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
1,1,0.351183,-0.064442,0.702367,-0.128885,0.702367,-0.128885,NaN
2,2,1.056750,-0.049547,0.708769,0.158676,0.006403,0.287561,NaN
3,3,2.159170,0.023684,1.496070,-0.012214,0.787300,-0.170890,NaN
4,4,3.672400,0.003846,1.530380,-0.027463,0.034315,-0.015249,NaN


,simstep,step,rollout,state_0,state_1,state_2,state_3,state_4,state_5,cost_cum,Unnamed: 10
0,0,0,0,-0.098166,-0.017666,-0.196333,-0.035332,-0.196333,-0.035332,223.703,NaN
1,0,0,1,0.032678,-0.035856,0.065355,-0.071713,0.065355,-0.071713,223.594,NaN
2,0,0,2,0.006231,0.007033,0.012463,0.014066,0.012463,0.014066,223.598,NaN
3,0,0,3,0.075502,0.029372,0.151003,0.058744,0.151003,0.058744,223.526,NaN
4,0,0,4,-0.037668,-0.037027,-0.075335,-0.074055,-0.075335,-0.074055,223.657,NaN


In [136]:
#df_timestep = filter_df(sampling_df, "simstep", 10)
#df_timestep = filter_df(df_timestep, "step", 3)

In [137]:
def draw_interactive(simstep, step):
    df_timestep = filter_df(sampling_df, "simstep", simstep, op='==')
    df_timestep = filter_df(df_timestep, "step", step, op='<=')
    
    fig, ax = fig, ax = plt.subplots(1, figsize=(15,10))
   
    if config_dict['obstacle_cost'][0] > 0:
        for obstacle_nr in range(0,4):
            obst_id_rad = 'obstacle_rad_'+str(obstacle_nr)
            obst_id_x = 'obstacle_pos_'+str(obstacle_nr)+'_x'
            obst_id_y = 'obstacle_pos_'+str(obstacle_nr)+'_y'
            
            ellipse_plot_0 = patches.Circle((config_dict[obst_id_x][0],config_dict[obst_id_y][0]), 
                                     radius=config_dict[obst_id_rad][0], color='gray')
      
            ax.add_patch(ellipse_plot_0)
    
    ax.plot(sim_df[['state_0']][:simstep+1], sim_df[['state_1']][:simstep+1], c='r')
    ax.plot(sim_df[['state_0']], sim_df[['state_1']], c='r', alpha=0.3)
    ax.scatter(df_timestep[['state_0']], df_timestep[['state_1']], s=10, c='g',zorder=10)
    ax.scatter(sim_df[['state_0']].iloc[simstep], sim_df[['state_1']].iloc[simstep],s=60, c='r',zorder=20)


    padding = 10;
    ax.set_xlim(sim_df[['state_0']].min()[0]-padding, sim_df[['state_0']].max()[0]+padding)
    ax.set_ylim(sim_df[['state_1']].min()[0]-padding, sim_df[['state_1']].max()[0]+padding)
    plt.show()
    
widgets.interactive(draw_interactive, 
                    simstep = widgets.IntSlider(value=0, min=0, max=config_dict['sim_time'][0]-1, step=1), 
                    step= widgets.IntSlider(value=19, min=0, max=config_dict['horizon'][0]-1))

interactive(children=(IntSlider(value=0, description='simstep', max=149), IntSlider(value=14, description='ste…

In [11]:
sim_df[['state_0']].min()[0]

-3.01472